In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt

# হাইপারপ্যারামিটার
latent_dim = 100
img_size = 28 * 28
batch_size = 64
epochs = 5
lr = 0.0002

# ডেটালোডার (MNIST)
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])
train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)



# জেনারেটর নেটওয়ার্ক
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(latent_dim, 256),
            nn.LeakyReLU(0.2),
            nn.Linear(256, 512),
            nn.LeakyReLU(0.2),
            nn.Linear(512, img_size),
            nn.Tanh()
        )
    
    def forward(self, z):
        return self.model(z)

# ডিস্ক্রিমিনেটর নেটওয়ার্ক
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(img_size, 512),
            nn.LeakyReLU(0.2),
            nn.Linear(512, 256),
            nn.LeakyReLU(0.2),
            nn.Linear(256, 1),
            nn.Sigmoid()
        )
    
    def forward(self, img):
        return self.model(img.view(img.size(0), -1))

# মডেল, অপ্টিমাইজার, লস
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
G = Generator().to(device)
D = Discriminator().to(device)
optimizer_G = optim.Adam(G.parameters(), lr=lr)
optimizer_D = optim.Adam(D.parameters(), lr=lr)
loss_fn = nn.BCELoss()

# ট্রেনিং লুপ
for epoch in range(epochs):
    for i, (imgs, _) in enumerate(train_loader):
        # আসল এবং নকল লেবেল
        real_imgs = imgs.to(device)
        real_labels = torch.ones(imgs.size(0), 1).to(device)
        fake_labels = torch.zeros(imgs.size(0), 1).to(device)
        
        # ====== ডিস্ক্রিমিনেটর ট্রেন ======
        optimizer_D.zero_grad()
        
        # আসল ইমেজের লস
        real_output = D(real_imgs)
        d_loss_real = loss_fn(real_output, real_labels)
        
        # নকল ইমেজ জেনারেট
        z = torch.randn(imgs.size(0), latent_dim).to(device)
        fake_imgs = G(z)
        
        # নকল ইমেজের লস 
        fake_output = D(fake_imgs.detach())
        d_loss_fake = loss_fn(fake_output, fake_labels)
        
        # মোট লস এবং ব্যাকপ্রপ 
        d_loss = d_loss_real + d_loss_fake
        d_loss.backward()
        optimizer_D.step()
        
        # ====== জেনারেটর ট্রেন ====== 
        optimizer_G.zero_grad()
        
        # জেনারেটরের লস
        validity = D(fake_imgs)
        g_loss = loss_fn(validity, real_labels)
        
        g_loss.backward()
        optimizer_G.step()
        
        # প্রোগ্রেস প্রিন্ট 
        if i % 200 == 0:
            print(f"[Epoch {epoch}/{epochs}] [Batch {i}/{len(train_loader)}] "
                  f"[D loss: {d_loss.item()}] [G loss: {g_loss.item()}]")
    
    # জেনারেট করা ইমেজ সেভ 
    with torch.no_grad():
        sample_z = torch.randn(16, latent_dim).to(device)
        generated = G(sample_z).cpu()
        plt.figure(figsize=(4, 4))
        for i in range(16):
            plt.subplot(4, 4, i+1)
            plt.imshow(generated[i].view(28, 28).numpy(), cmap='gray')
            plt.axis('off')
        plt.savefig(f"gan_output_epoch_{epoch}.png")
        plt.close()


100%|██████████| 9.91M/9.91M [00:00<00:00, 60.1MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 1.70MB/s]
100%|██████████| 1.65M/1.65M [00:00<00:00, 14.7MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 6.10MB/s]


[Epoch 0/5] [Batch 0/938] [D loss: 1.3919553756713867] [G loss: 0.6825375556945801]
[Epoch 0/5] [Batch 200/938] [D loss: 0.5505620241165161] [G loss: 2.2923526763916016]
[Epoch 0/5] [Batch 400/938] [D loss: 0.09082544595003128] [G loss: 4.3156023025512695]
[Epoch 0/5] [Batch 600/938] [D loss: 0.8928648233413696] [G loss: 1.957014560699463]
[Epoch 0/5] [Batch 800/938] [D loss: 0.3390049934387207] [G loss: 2.895840883255005]
[Epoch 1/5] [Batch 0/938] [D loss: 0.4340571165084839] [G loss: 3.332082509994507]
[Epoch 1/5] [Batch 200/938] [D loss: 0.7202643156051636] [G loss: 1.9506603479385376]
[Epoch 1/5] [Batch 400/938] [D loss: 0.599041759967804] [G loss: 4.4672393798828125]
[Epoch 1/5] [Batch 600/938] [D loss: 1.322713851928711] [G loss: 1.6873303651809692]
[Epoch 1/5] [Batch 800/938] [D loss: 0.2985282242298126] [G loss: 3.277575969696045]
[Epoch 2/5] [Batch 0/938] [D loss: 1.0805976390838623] [G loss: 1.3734402656555176]
[Epoch 2/5] [Batch 200/938] [D loss: 1.285544514656067] [G loss: 

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt

In [4]:
# হাইপারপ্যারামিটার
latent_dim = 100
img_size = 28 * 28
batch_size = 64
epochs = 5
lr = 0.0002

# ডেটালোডার (MNIST)
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])
train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

In [5]:
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        self.model=nn.Sequential(
            nn.Linear(latent_dim,256),
            nn.LeakyReLU(0.2),
            nn.Linear(256,512),
            nn.LeakyReLU(0.2),
            nn.Linear(512,img_size),
            nn.Tanh()
        )
    def foreward(self, z):
        return self.model(z)


class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        nn.model=nn.Sequential(
            nn.Linear(img_size, 256),
            nn.LeakyReLU(0.2),
            nn.Linear(256,512),
            nn.LeakyReLU(0.2),
            nn.Linear(512,1),
            nn.Sigmoid()
        )
        def foreward(self, img):
            self.model(img.view(img.size(0), -1))


    
    
    

In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_Gen=Generator().to(device)
model_Dis=Discriminator().to(device)